In [1]:
# %% 
import numpy as np
from sklearn.model_selection import train_test_split
#%%
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

print(tf.__version__)

2.16.1


In [2]:

def grad_loss(v_gt, v):
    # Gradient loss
    loss = tf.reduce_mean(tf.abs(v - v_gt), axis=[1,2,3])
    jy = v[:,:,1:,:,:] - v[:,:,:-1,:,:]
    jx = v[:,:,:,1:,:] - v[:,:,:,:-1,:]
    jy_ = v_gt[:,:,1:,:,:] - v_gt[:,:,:-1,:,:]
    jx_ = v_gt[:,:,:,1:,:] - v_gt[:,:,:,:-1,:]
    loss += tf.reduce_mean(tf.abs(jy - jy_), axis=[1,2,3])
    loss += tf.reduce_mean(tf.abs(jx - jx_), axis=[1,2,3])
    return loss

In [3]:
def uNet(input, time, lat, lon, height, kernel = [5, 3, 3], nodes = [72, 144, 288, 576]):
    '''
    This function defines a U-Net architecture
    :param input: the main-input layer
    :param time: the time input layer
    :param lat, lon, height: additional fields
    :param kernel: Kernel-sizes (default = [5, 3, 3])
    :param nodes: different neuron-sizes if needed (default = [72, 144, 288, 576])
    :return: last layer of constructed model
    '''

    # set Timesteps
    TS = 3
    ##################################################### 1st Block ####################################################
    conv1 = Conv3D(filters      = nodes[0],
                   kernel_size  = (TS, kernel[0], kernel[0]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(input)
    mergetime = Concatenate(axis=4)([conv1, lat, lon, height])
    conv1 = Conv3D(filters      = nodes[0],
                   kernel_size  = kernel[0],
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(mergetime)

    pool1 = MaxPooling3D(pool_size = (1, 2, 2))(conv1)

    ##################################################### 2nd Block ####################################################
    conv2 = Conv3D(filters      = nodes[1],
                   kernel_size  = (TS, kernel[1], kernel[1]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool1)
    conv2 = Conv3D(filters      = nodes[1],
                   kernel_size  = (TS, kernel[1], kernel[1]),
                   activation   = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv2)

    pool2 = MaxPooling3D(pool_size = (1, 2, 2))(conv2)

    ##################################################### 3rd Block ####################################################
    conv3 = Conv3D(filters      = nodes[2],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation = 'relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool2)
    conv3 = Conv3D(filters      = nodes[2],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv3)

    pool3 = MaxPooling3D(pool_size = (1, 2, 2))(conv3)

    ##################################################### 4th Block ####################################################
    conv4 = Conv3D(filters      = nodes[3],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(pool3)
    conv4 = Conv3D(filters      = nodes[3],
                   kernel_size  = (TS, kernel[2], kernel[2]),
                   activation='relu',
                   padding      = 'same',
                   data_format  = 'channels_last')(conv4)

    ####################################################### TIME #######################################################
    # Merge time-layer at this point
    mergetime = Concatenate(axis=4)([conv4, time])

    ################################################### UP 3rd Block ###################################################
    # Up-Size again
    up3   = UpSampling3D(size = (1, 2, 2))(mergetime)
    up3   = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   kernel_initializer   = 'he_normal')(up3)

    # Skip connection
    merge3 = Concatenate(axis=4)([conv3, up3])

    conv3 = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge3)
    conv3 = Conv3D(filters              = nodes[2],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv3)

    ################################################### UP 2nd Block ###################################################
    up2 = UpSampling3D(size = (1, 2, 2))(conv3)
    up2 = Conv3D(filters                = nodes[1],
                 kernel_size            = (TS, kernel[1], kernel[1]),
                 activation             = 'relu',
                 padding                = 'same',
                 kernel_initializer     = 'he_normal')(up2)

    # Skip connection
    merge2 = Concatenate(axis=4)([conv2, up2])

    conv2 = Conv3D(filters              = nodes[1],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge2)
    conv2 = Conv3D(filters              = nodes[1],
                   kernel_size          = (TS, kernel[1], kernel[1]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv2)

    ################################################### UP 1st Block ###################################################
    up1 = UpSampling3D(size = (1, 2, 2))(conv2)
    up1 = Conv3D(filters                = nodes[0],
                 kernel_size            = (TS, kernel[0], kernel[0]),
                 activation             = 'relu',
                 padding                = 'same',
                 kernel_initializer     = 'he_normal')(up1)

    merge1 = Concatenate(axis=4)([conv1, up1])

    conv1 = Conv3D(filters              = nodes[0],
                   kernel_size          = (TS, kernel[0], kernel[0]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(merge1)
    conv1 = Conv3D(filters              = nodes[0],
                   kernel_size          = (TS, kernel[0], kernel[0]),
                   activation           = 'relu',
                   padding              = 'same',
                   data_format          = 'channels_last')(conv1)

    # last layer is the output
    output = conv1

    return output


In [4]:
def get_model(PS=32, loss = grad_loss, optimizer = 'adam', nodes = [72, 144, 288, 576], residual = False):
    '''
    This function creates the DCN-architecture (residual = False) or RPN-architecture (residual = True).
    :param PS: Patch size
    :param loss: loss function (default = grad_loss)
    :param optimizer: optimizer (default = 'adam')
    :param nodes: different neuron-sizes if needed (default = [72, 144, 288, 576])
    :param residual: boolean toggeling between RPN (True) and DCN (False)
    :return: Model
    '''

    # Input layers
    main_input  = Input(shape = (3, PS, PS, 1))
    time        = Input(shape = (3, int(PS/8), int(PS/8), 1))
    lat         = Input(shape = (3, PS, PS, 1))
    lon         = Input(shape = (3, PS, PS, 1)) 
    height      = Input(shape = (3, PS, PS, 1))

    # Load U-Net
    unet        = uNet(main_input, time, lat, lon, height, nodes = nodes)

    # Define output layer after U-Net
    temp_out    = Conv3D(filters        = 1,
                         kernel_size    = (3, 1, 1),
                         activation     = 'linear',
                         padding        = 'valid',
                         data_format    = "channels_last")(unet)
    
    # # Define output layer after U-Net
    # temp_out    = Conv3D(filters        = 5,
    #                      kernel_size    = (1, 1, 1),
    #                      activation     = 'linear',
    #                      padding        = 'valid',
    #                      data_format    = "channels_first")(temp_out)
    
    # residual layer
    if residual:
        temp_out = Add()([main_input[:,1,:,:], temp_out])

    # create model with the defined Layers
    model       = Model(inputs          = [main_input, time, lat, lon, height],
                        outputs         = temp_out)

    # compile with defined loss and optimizer
    model.compile(loss      = loss,
                  optimizer = optimizer,
                  metrics   = ['mse', 'mae', 'mape'])

    return model

def main():
  model = get_model() # DCN
#   model = get_model(residual=True) # RPN
  model.summary()
  print(model.summary())

We want data in the size of patch_size x patch_size in time step of 3

In [5]:
# Load data
wind_speed_data = np.load('tensor.npy')  # Assuming wind speed is in the first channel
print(wind_speed_data.shape)
wind_speed_data = wind_speed_data[:, :, :, 0]  # Extract wind speed channel

# Normalize the data
wind_speed_data = (wind_speed_data - np.min(wind_speed_data)) / (np.max(wind_speed_data) - np.min(wind_speed_data))

(52704, 23, 34, 2)


In [6]:
wind_speed_data.shape

(52704, 23, 34)

In [7]:
# %%
# Reshape data to fit the model input shape
def reshape_data(data, patch_size=32, time_steps=3):
    # Define the target shape
    target_shape = (32, 32)
    data = data[..., np.newaxis]
    # Use tf.image.resize to resize the images
    resized_data = tf.image.resize(data, target_shape, method='bilinear')

    # Convert back to NumPy array if needed
    data = resized_data.numpy()


    num_samples = data.shape[0] - time_steps + 1
    patches = []
    time_patches = []
    relative_time = np.zeros((3, 4, 4, 1))

    # Assign specific values to each slice along the first axis
    relative_time[0, :, :, 0] = -1
    relative_time[1, :, :, 0] = 0
    relative_time[2, :, :, 0] = 1
    for i in range(num_samples):
        # print(i, i +time_steps)
        patch = data[i:i+time_steps, :, :32]
        patches.append(patch)
        time_patches.append(relative_time)
    patches = np.array(patches)
    time_patches = np.array(time_patches)
    print(patches.shape)
    print(time_patches.shape)
    # patches = np.expand_dims(patches, axis=-1)  # Add channel dimension
    return patches, time_patches

# Example reshape
patch_size = 32
time_steps = 3
wind_speed_patches, time_patches = reshape_data(wind_speed_data, patch_size, time_steps)

(52702, 3, 32, 32, 1)
(52702, 3, 4, 4, 1)


In [8]:
# import numpy as np
# import tensorflow as tf

# # Reshape data to fit the model input shape
# def reshape_data(data, target_shape=(32, 32)):
#     # Reshape data to the target shape
#     data = data[..., np.newaxis]
#     resized_data = tf.image.resize(data, target_shape, method='bilinear')
#     data = resized_data.numpy()
#     return data

# def create_patches(data, time_steps):
#     num_samples = data.shape[0] - time_steps + 1
#     patches = []

#     relative_time = np.zeros((time_steps, data.shape[1]//8, data.shape[2]//8, 1))
#     # relative_time = np.zeros((3, 4, 4, 1))

#     # Assign specific values to each slice along the first axis
#     # relative_time[0, :, :, 0] = -1
#     # relative_time[1, :, :, 0] = 0
#     # relative_time[2, :, :, 0] = 1
#     for t in range(time_steps):
#         relative_time[t, :, :, 0] = t

#     for i in range(num_samples):
#         patch = data[i:i+time_steps]
#         patches.append(patch)

#     patches = np.array(patches)
#     relative_time_patches = np.tile(relative_time, (num_samples, 1, 1, 1, 1))

#     return patches, relative_time_patches


# # Parameters
# patch_size = 32
# time_steps_input = 3
# time_steps_label = 5

# # Reshape data
# reshaped_data = reshape_data(wind_speed_data, target_shape=(32, 32))

# # Create training input patches with time_steps_input
# input_patches, input_time_patches = create_patches(reshaped_data, time_steps_input)

# # Create label patches with time_steps_label
# label_patches, label_time_patches = create_patches(reshaped_data, time_steps_label)

# input_patches, input_time_patches = input_patches[:label_patches.shape[0],:,:,:,:], input_time_patches[:label_patches.shape[0],:,:,:,:]

# print("Input patches shape:", input_patches.shape)
# print("Input time patches shape:", input_time_patches.shape)
# print("Label patches shape:", label_patches.shape)
# print("Label time patches shape:", label_time_patches.shape)


In [9]:
import numpy as np
import tensorflow as tf

# Reshape data to fit the model input shape
def reshape_data(data, target_shape=(32, 32)):
    # Reshape data to the target shape
    data = data[..., np.newaxis]
    resized_data = tf.image.resize(data, target_shape, method='bilinear')
    data = resized_data.numpy()
    return data

def create_patches(data, time_steps):
    num_samples = data.shape[0] - time_steps + 1
    training_patches = []
    patches = []

    

    for i in range(num_samples):
        patch = data[i:i+time_steps]
        patches.append(patch)
        training_patches.append(patch[::2])
    training_timestep = time_steps//2+1
    relative_time = np.zeros((time_steps//2+1, data.shape[1]//8, data.shape[2]//8, 1))
    # relative_time = np.zeros((3, 4, 4, 1))

    # Assign specific values to each slice along the first axis
    # relative_time[0, :, :, 0] = -1
    # relative_time[1, :, :, 0] = 0
    # relative_time[2, :, :, 0] = 1
    for t in range(training_timestep):
        relative_time[t, :, :, 0] = t
    relative_time_patches = np.tile(relative_time, (num_samples, 1, 1, 1, 1))
    training_patches = np.array(training_patches)
    patches = np.array(patches)

    return training_patches, relative_time_patches, patches


# Parameters
patch_size = 32
time_steps_label = 5

# Reshape data
reshaped_data = reshape_data(wind_speed_data, target_shape=(32, 32))

# Create label patches with time_steps_label
input_patches, input_time_patches, label_patches = create_patches(reshaped_data, time_steps_label)

# input_patches, input_time_patches = input_patches[:label_patches.shape[0],:,:,:,:], input_time_patches[:label_patches.shape[0],:,:,:,:]

print("Input patches shape:", input_patches.shape)
print("Input time patches shape:", input_time_patches.shape)
print("Label patches shape:", label_patches.shape)



Input patches shape: (52700, 3, 32, 32, 1)
Input time patches shape: (52700, 3, 4, 4, 1)
Label patches shape: (52700, 5, 32, 32, 1)


In [10]:
# # Split data
# train_inputs, val_inputs, train_inputs_time, val_inputs_time = train_test_split(
#     input_patches, input_time_patches, test_size=0.2, random_state=42)

# train_labels, val_labels, train_labels_time, val_labels_time = train_test_split(
#     label_patches, test_size=0.2, random_state=42)
# print("Training input patches shape:", train_inputs.shape)
# print("Training input time patches shape:", train_inputs_time.shape)
# print("Val input time patches shape:", val_inputs_time.shape)
# print("Validation input patches shape:", val_inputs.shape)
# print("Training label patches shape:", train_labels.shape)
# print("Validation label patches shape:", val_labels.shape)

In [11]:
# Define the size of the training set
train_size = int(0.8 * len(input_patches))  # 80% for training

# Create indices
indices = np.arange(len(input_patches))
np.random.seed(42)  # Set seed for reproducibility
np.random.shuffle(indices)

# Split indices into training and validation
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Use the indices to split the data
train_inputs = input_patches[train_indices]
val_inputs = input_patches[val_indices]

train_inputs_time = input_time_patches[train_indices]
val_inputs_time = input_time_patches[val_indices]

train_labels = label_patches[train_indices]
val_labels = label_patches[val_indices]


In [12]:
train_labels.shape

(42160, 5, 32, 32, 1)

In [13]:
# # %%
# # Train the model
# epochs = 5
# batch_size = 32
# model = get_model(PS=batch_size)
# # history = model.fit([train_inputs, train_inputs_time, train_inputs, train_inputs, train_inputs], 
# #                     train_labels,
# #                     validation_data=([val_inputs, val_inputs, val_inputs, val_inputs, val_inputs], val_labels),
# #                     epochs=epochs,
# #                     batch_size=batch_size)
# history = model.fit([train_inputs, train_inputs_time, train_inputs, train_inputs, train_inputs], 
#                     train_labels, verbose=1)

In [14]:
from keras.callbacks import ModelCheckpoint

# Define the model
epochs = 50
batch_size = 32
model = get_model(PS=batch_size)

# Define the checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='best_model.keras',  # Filepath where the model will be saved
    monitor='val_loss',        # Monitor the validation loss
    save_best_only=True,       # Save only the best model
    save_weights_only=False,   # Save the whole model, not just weights
    mode='min',                # Mode 'min' means it will save the model with the minimum validation loss
    verbose=1                  # Verbosity mode
)

# Train the model with validation and checkpointing
history = model.fit(
    [train_inputs, train_inputs_time, train_inputs, train_inputs, train_inputs],
    train_labels,
    validation_data=([val_inputs, val_inputs, val_inputs, val_inputs, val_inputs], val_labels),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[checkpoint_callback],
    verbose=1
)


Epoch 1/50
1318/1318 ━━━━━━━━━━━━━━━━━━━━ 0s 18s/step - loss: 0.1722 - mae: 0.1516 - mape: 69.6839 - mse: 4.1816 

ValueError: Input 1 of layer "functional" is incompatible with the layer: expected shape=(None, 3, 4, 4, 1), found shape=(None, 3, 32, 32)

In [ ]:
# def uNet(input, time, lat, lon, height, kernel = [5, 3, 3], nodes = [72, 144, 288, 576]):
#     '''
#     This function defines a U-Net architecture
#     :param input: the main-input layer
#     :param time: the time input layer
#     :param lat, lon, height: additional fields
#     :param kernel: Kernel-sizes (default = [5, 3, 3])
#     :param nodes: different neuron-sizes if needed (default = [72, 144, 288, 576])
#     :return: last layer of constructed model
#     '''

#     # set Timesteps
#     TS = 3
#     ##################################################### 1st Block ####################################################
#     conv1 = Conv3D(filters      = nodes[0],
#                    kernel_size  = (TS, kernel[0], kernel[0]),
#                    activation   = 'relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(input)
#     mergetime = Concatenate(axis=4)([conv1, lat, lon, height])
#     conv1 = Conv3D(filters      = nodes[0],
#                    kernel_size  = kernel[0],
#                    activation   = 'relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(mergetime)

#     pool1 = MaxPooling3D(pool_size = (1, 2, 2))(conv1)

#     ##################################################### 2nd Block ####################################################
#     conv2 = Conv3D(filters      = nodes[1],
#                    kernel_size  = (TS, kernel[1], kernel[1]),
#                    activation   = 'relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(pool1)
#     conv2 = Conv3D(filters      = nodes[1],
#                    kernel_size  = (TS, kernel[1], kernel[1]),
#                    activation   = 'relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(conv2)

#     pool2 = MaxPooling3D(pool_size = (1, 2, 2))(conv2)

#     ##################################################### 3rd Block ####################################################
#     conv3 = Conv3D(filters      = nodes[2],
#                    kernel_size  = (TS, kernel[2], kernel[2]),
#                    activation = 'relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(pool2)
#     conv3 = Conv3D(filters      = nodes[2],
#                    kernel_size  = (TS, kernel[2], kernel[2]),
#                    activation='relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(conv3)

#     pool3 = MaxPooling3D(pool_size = (1, 2, 2))(conv3)

#     ##################################################### 4th Block ####################################################
#     conv4 = Conv3D(filters      = nodes[3],
#                    kernel_size  = (TS, kernel[2], kernel[2]),
#                    activation='relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(pool3)
#     conv4 = Conv3D(filters      = nodes[3],
#                    kernel_size  = (TS, kernel[2], kernel[2]),
#                    activation='relu',
#                    padding      = 'same',
#                    data_format  = 'channels_last')(conv4)

#     ####################################################### TIME #######################################################
#     # Merge time-layer at this point
#     mergetime = Concatenate(axis=4)([conv4, time])

#     ################################################### UP 3rd Block ###################################################
#     # Up-Size again
#     up3   = UpSampling3D(size = (1, 2, 2))(mergetime)
#     up3   = Conv3D(filters              = nodes[2],
#                    kernel_size          = (TS, kernel[1], kernel[1]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    kernel_initializer   = 'he_normal')(up3)

#     # Skip connection
#     merge3 = Concatenate(axis=4)([conv3, up3])

#     conv3 = Conv3D(filters              = nodes[2],
#                    kernel_size          = (TS, kernel[1], kernel[1]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(merge3)
#     conv3 = Conv3D(filters              = nodes[2],
#                    kernel_size          = (TS, kernel[1], kernel[1]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(conv3)

#     ################################################### UP 2nd Block ###################################################
#     up2 = UpSampling3D(size = (1, 2, 2))(conv3)
#     up2 = Conv3D(filters                = nodes[1],
#                  kernel_size            = (TS, kernel[1], kernel[1]),
#                  activation             = 'relu',
#                  padding                = 'same',
#                  kernel_initializer     = 'he_normal')(up2)

#     # Skip connection
#     merge2 = Concatenate(axis=4)([conv2, up2])

#     conv2 = Conv3D(filters              = nodes[1],
#                    kernel_size          = (TS, kernel[1], kernel[1]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(merge2)
#     conv2 = Conv3D(filters              = nodes[1],
#                    kernel_size          = (TS, kernel[1], kernel[1]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(conv2)

#     ################################################### UP 1st Block ###################################################
#     up1 = UpSampling3D(size = (1, 2, 2))(conv2)
#     up1 = Conv3D(filters                = nodes[0],
#                  kernel_size            = (TS, kernel[0], kernel[0]),
#                  activation             = 'relu',
#                  padding                = 'same',
#                  kernel_initializer     = 'he_normal')(up1)

#     merge1 = Concatenate(axis=4)([conv1, up1])

#     conv1 = Conv3D(filters              = nodes[0],
#                    kernel_size          = (TS, kernel[0], kernel[0]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(merge1)
#     conv1 = Conv3D(filters              = nodes[0],
#                    kernel_size          = (TS, kernel[0], kernel[0]),
#                    activation           = 'relu',
#                    padding              = 'same',
#                    data_format          = 'channels_last')(conv1)

#     # last layer is the output
#     output = conv1

#     return output


In [ ]:
# # Input layers
# PS = 32
# main_input  = Input(shape = (3, PS, PS, 1))
# time        = Input(shape = (3, int(PS/8), int(PS/8), 1))
# lat         = Input(shape = (3, PS, PS, 1))
# lon         = Input(shape = (3, PS, PS, 1)) 
# height      = Input(shape = (3, PS, PS, 1))